In [8]:
# Install Selenium and WebDriver Manager
!pip install selenium
!pip install webdriver-manager
!pip install tqdm

# Install Chrome
!apt-get update
!apt-get install -y wget unzip
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Check the versions of Chrome
!google-chrome --version


zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
zsh:1: command not found: wget
zsh:1: command not found: dpkg
zsh:1: command not found: apt-get
zsh:1: command not found: google-chrome


In [10]:
# MacOS Version
# Install Selenium and WebDriver Manager
!pip install selenium
!pip install webdriver-manager
!pip install tqdm

# Install Homebrew if not installed
!which brew || /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

# Install Chrome using Homebrew
!brew install --cask google-chrome

# Check the version of Chrome
!"/Applications/Google Chrome.app/Contents/MacOS/Google Chrome" --version


/opt/homebrew/bin/brew
==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://dl.google.com/chrome/mac/universal/stable/GGRO/googlechr
######################################################################### 100.0%
==> Installing Cask google-chrome
==> Purging files for version 125.0.6422.113 of Cask google-chrome
Error: It seems there is already an App at '/Applications/Google Chrome.app'.
Google Chrome 125.0.6422.78 


In [52]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import requests
from tqdm import tqdm
import random
import os



In [2]:
# Lib for image recognition
import pytesseract
from PIL import Image
from io import BytesIO


In [3]:
# define driver
def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    # options.add_argument('--disable-gpu')
    # options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920x1080')
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    chrome_options.add_argument("accept=text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8")
    chrome_options.add_argument("accept-language=en-US,en;q=0.9")
    chrome_options.add_argument("accept-encoding=gzip, deflate, br")
    chrome_options.add_argument("upgrade-insecure-requests=1")
    chrome_options.add_argument("cache-control=no-cache")
    # driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options) # if error, use this version
    return driver



In [18]:
# Function to perform random sleep to mimic human behavior
def random_sleep(min_seconds, max_seconds):
    sleep(random.uniform(min_seconds, max_seconds))

def debug_page(filename_prefix):
    if not os.path.exists('debug_dir'):
        os.makedirs('debug_dir')
    page_source = driver.page_source
    with open(f"debug_dir/{filename_prefix}_page_source.html", "w", encoding="utf-8") as file:
        file.write(page_source)
    driver.save_screenshot(f"debug_dir/{filename_prefix}_screenshot.png")
    print(f"Debug info saved: {filename_prefix}_page_source.html and {filename_prefix}_screenshot.png")

# Function to handle the Terms of Use page
def handle_terms_of_use():
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        accept_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, 'ctl00_mainContentArea_disclaimerContent_yesButton'))
        )
        # debug_page("terms_of_use")
        accept_button.click()
        random_sleep(1, 2)  # Wait a bit to ensure the page loads completely
        # print("Accepted Terms of Use")
        return True
    except Exception as e:
        print(f"No Terms of Use page found or unable to click accept button: ")
        # debug_page("terms_of_use_error")
    return False

## Scraping Search Results

In [68]:
def select_date(input_id, date):
    month = date.strftime("%b")
    year = date.strftime("%Y")
    day = date.strftime("%d").lstrip("0")

    input_element = driver.find_element(By.ID, input_id)
    if input_element:
      input_element.click()
    # random_sleep(1, 2)
    # debug_page(f"select_date_{input_id}")

    year_selector = Select(driver.find_element(By.CLASS_NAME, 'ui-datepicker-year'))
    if year_selector:
      year_selector.select_by_visible_text(year)
    # debug_page(f"select_date_year_{input_id}")
    month_selector = Select(driver.find_element(By.CLASS_NAME, 'ui-datepicker-month'))
    if month_selector:
      month_selector.select_by_visible_text(month)
    # debug_page(f"select_date_month_{input_id}")
    day_element = driver.find_element(By.XPATH, f"//a[@data-date='{day}']")
    if day_element:
      day_element.click()
    # debug_page(f"select_date_day_{input_id}")

# Function to select the necessary checkboxes
def select_filings(filings):
    # for filing in filings:
    #   try:
    #       label = driver.find_element(By.XPATH, f'//label[text()="{filing}"]')
    #       checkbox = label.find_element(By.XPATH, './preceding-sibling::input[@type="checkbox"]')
    #       # print(label.text)
    #       checkbox.click()
    #       # print(f"Selected filing: {filing}")
    #   except Exception as e:
    #       print(f"Unable to select filing {filing}: {e}")
    #       debug_page(f"select_filing_error_{filing}")
    
    # for one checkbox only
    try:
        label = driver.find_element(By.XPATH, f'//label[text()="{filings}"]')
        checkbox = label.find_element(By.XPATH, './preceding-sibling::input[@type="checkbox"]')
        # print(label.text)
        checkbox.click()
        # print(f"Selected filing: {filing}")
    except Exception as e:
        print(f"Unable to select filing {filings}: {e}")
        debug_page(f"select_filing_error_{filings}")

# Function to perform the search
def perform_search(start_date, end_date, filings):
  print('=' * 50)
  print(f"scraping {filings} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

  try:
      driver.get('https://emma.msrb.org/Search/Search.aspx')
      handle_terms_of_use()  # Call the function to handle Terms of Use

      # Click on the Disclosures tab
      try:
          disclosures_tab = WebDriverWait(driver, 2).until(
              EC.element_to_be_clickable((By.ID, 'disclosuresFilterLi'))
          )
          disclosures_tab.click()
          # print('clicked on disclosures tab')
          # random_sleep(2, 4)
          # debug_page("disclosures_tab")
      except Exception as e:
          print(f"Unable to locate or click the Disclosures tab")
          # debug_page("disclosures_tab_error")
          print('try terms_of_use')
          if handle_terms_of_use():
            df = perform_search(start_date, end_date, filings)
            return df
          else:
            print('=' * 50)
            print('error when handling terms of use')
            return
      # Input the start and end dates

      select_date('postingDateFrom', start_date)
      select_date('postingDateTo', end_date)

      # Select the necessary checkboxes
      select_filings(filings)
      
      # debug_page("checkbox")
      # Click the "Run Search" button
      run_search_button = driver.find_element(By.ID, 'runSearchButton')
      run_search_button.click()
      random_sleep(2, 4)
      # Wait for the counter label to appear
      search_results_loaded = WebDriverWait(driver, 30).until(
          EC.presence_of_element_located((By.CLASS_NAME, 'searchResultsSecurityView'))
      )

      print('*** page loaded')
      # debug_page("run_search")
      length_selector = Select(WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.NAME, 'lvDocuments_length'))
      ))
      length_selector.select_by_value('100')
      # debug_page("length_select")
      counter_label = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.ID, 'counterLabel'))
      )
      num_disclosures = counter_label.text.split()[0]
      print(f"Number of disclosures: {num_disclosures}")
      if num_disclosures == '0':
          # print(f"No results found from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
          return None
      df = scrape_results(num_disclosures)
      if not os.path.exists(f'results/{filings}'):
        os.makedirs(f'results/{filings}')
      filename = f"results/{filings}/results_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}.csv"
      df.to_csv(filename, index=False)
      print('*'*10, filename, 'Saved')
      return df
  except Exception as e:
      print('*'*10, f"Error during perform_search from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
      debug_page(f"perform_search_error_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}")
      return None


In [57]:
# Function to scrape the search results
def scrape_results(num_disclosures):
    results = []
    page = 1
    try:
        pbar = tqdm(total=int(num_disclosures), desc='Scraping results', unit='rows')
        while True:
            print('scraping page: ', page)
            WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.ID, 'lvDocuments_wrapper'))
            )
            # debug_page("scrape_results")

            # Scrape the search results from the current page
            rows = driver.find_elements(By.CSS_SELECTOR, '#lvDocuments tbody tr')
            # print('Number of rows: ', len(rows))
            for row in rows:
                issuer_name = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text
                disclosure_desc = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a').text
                link = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a').get_attribute('href')
                date_posted = row.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text
                results.append({
                    'issuer_name': issuer_name,
                    'disclosure_desc': disclosure_desc,
                    'link': link,
                    'date_posted': date_posted
                })
                # print(f"Issuer: {issuer_name}, Description: {disclosure_desc}, Link: {link}, Date: {date_posted}")

            pbar.update(len(rows))
            # print(f"{(page - 1)* 100 + len(rows)} / {num_disclosures} Saved")

            # Check if there is a next page
            next_button = driver.find_element(By.ID, 'lvDocuments_next')
            if "disabled" in next_button.get_attribute("class"):
                break  # Exit loop if next button is disabled

            next_button.click()  # Go to the next page
            page += 1
            random_sleep(1, 2)  # Wait for the next page to load

    except Exception as e:
        print(f"Error during scraping results: {e}")
        debug_page("scrape_results_error")
    pbar.close()
    print("\n")
    return pd.DataFrame(results)


In [70]:
filings = [
    "Annual Financial Information and Operating Data",
    "Audited Financial Statements or ACFR",
    "Budget",
    "Change in Accounting Standard",
    "Change in Fiscal Year / Timing of Annual Disclosure",
    "Consultant Reports",
    "Failure to Provide Annual Financial Information as Required",
    "Information Provided to Rating Agency, Credit / Liquidity Provider or Other Third Party",
    "Interim / Additional Financial Information / Operating Data",
    "Investment / Debt / Financial Policy",
    "Other Financial / Operating Data",
    "Quarterly / Monthly Financial Information"
]

# input date range and filings
start_date = datetime(2024, 4, 24)
end_date = datetime(2024, 4, 26)
range = 1 # zero based, 1 means 2 days
filing = filings[7] # select one filing


# run the scraper
df_list = []
while start_date <= end_date:
  driver = setup_driver()
  mid_date = start_date + timedelta(days=range) if start_date + timedelta(days=range) < end_date else end_date
  df = perform_search(start_date, mid_date, filing)
  if df is not None:
    df_list.append(df)
  else:
    print(f"No results found from {start_date.strftime('%Y-%m-%d')} to {mid_date.strftime('%Y-%m-%d')}")
    
  start_date = mid_date + timedelta(days=1)
  driver.quit()
if df_list and all(df is not None for df in df_list):
  df = pd.concat(df_list)
  # df.to_csv(f"results/{filing}_combined.csv", index=False)
  print(len(df))

scraping Information Provided to Rating Agency, Credit / Liquidity Provider or Other Third Party from 2024-04-24 to 2024-04-25
*** page loaded
Number of disclosures: 2


Scraping results: 100%|██████████| 2/2 [00:00<00:00, 29.95rows/s]

scraping page:  1


********** results/Information Provided to Rating Agency, Credit / Liquidity Provider or Other Third Party/results_2024-04-24_2024-04-25.csv Saved


scraping Information Provided to Rating Agency, Credit / Liquidity Provider or Other Third Party from 2024-04-26 to 2024-04-26
*** page loaded
Number of disclosures: 0
No results found from 2024-04-26 to 2024-04-26
2


## Scrape detailed page

In [73]:
def read_csv(file_path):
    return pd.read_csv(file_path)

def download_pdf(url, file_name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/pdf',
        'Referer': 'https://emma.msrb.org/' 
    }
    
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status() 
        if not os.path.exists('downloaded_pdfs'):
            os.makedirs('downloaded_pdfs')
        download_dir = os.path.join('downloaded_pdfs', file_name)
        with open(download_dir, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {file_name} successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {file_name}. Error: {e}")
    except IOError as e:
        print(f"Failed to save {file_name}. Error: {e}")
        
# get cusip text
def get_cusip_text(img_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/pdf',
        'Referer': 'https://emma.msrb.org/' 
    }
    img_response = requests.get(img_url, headers=headers, stream=True)
    img = Image.open(BytesIO(img_response.content))
    cusip_text = pytesseract.image_to_string(img, config='--psm 7').strip()
    return cusip_text

# Scrape detail page
def scrape_detail_page(link):
    try:
        driver.get(link)
        
        # Handle the terms of use if it appears
        try:
            terms_of_use_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Municipal Securities Rulemaking Board's Website Terms of Use')]"))
            )
            if terms_of_use_element: 
                handle_terms_of_use()
        except Exception as e:
            print(f"No terms of use found")
        print('term_of_use handled')
        # Wait for the detail page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'card-body'))
        )
        print('find body')
        # click on the page to clear instructions
        body = driver.find_element(By.TAG_NAME, 'body')
        ActionChains(driver).move_to_element(body).click().perform()
        print('clicked')
        random_sleep(2, 4)
        # Scrape the required information
        
        '''
        Part one
        '''
        # Extract Filing Type
    
        filing_type = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, 'discType'))
        ).text.strip()
        
        # # Extract Disclosure Details
        # rule_mandated_div = WebDriverWait(driver, 5).until(
        #     EC.visibility_of_element_located((By.ID, 'ruleMandatedDiv'))
        # ).text.strip()
        
        # # Extract Posted Date and Associated CUSIPs
        # left_blue_box = WebDriverWait(driver, 5).until(
        #     EC.visibility_of_element_located((By.ID, 'leftBlueBoxDiv'))
        # ).text
        # posted_date = left_blue_box.split("Posted Date:")[1].split("Associated CUSIPs:")[0].strip()
        # associated_cusips = left_blue_box.split("Associated CUSIPs:")[1].strip()

        # disclosure_details = f"{rule_mandated_div}\nPosted Date: {posted_date}\nAssociated CUSIPs: {associated_cusips}"
        disclosure_details_div = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, 'leftBlueBoxDiv'))
        )   
        disclosure_details = disclosure_details_div.text
        
        print('disclosure_details:', disclosure_details)
        # Extract Contact Information
        contact_info_div = WebDriverWait(driver, 5).until(
            EC.any_of(
                # EC.visibility_of_element_located((By.ID, 'issuerContactDiv')),
                # EC.visibility_of_element_located((By.ID, 'submitterContactDiv')),
                EC.visibility_of_element_located((By.ID, 'rightBlueBoxDiv'))
            )
        )   
        print(contact_info_div.text)
        contact_info = contact_info_div.text
        # contact_info = ' '.join([elem.text.strip() for elem in contact_info_div.find_elements(By.TAG_NAME, 'td')])
        print('part one done')
        '''
        Part two
        '''
        # Extract Document Link
        # document_button = driver.find_element(By.XPATH, "//a[@id='viewDoc']")
        # document_link = document_button.get_attribute('href')
        # print(document_link)
        # document_name = document_link.split('/')[-1]
        
        # # download pdf
        # download_pdf(document_link, document_name)
        
        # Extract Document Link
        document_links = []
        document_names = []
        
        document_button = driver.find_element(By.XPATH, "//a[@id='viewDoc']")

        # Check if the document button has a 'href' attribute
        if document_button.get_attribute('href'):
            document_link = document_button.get_attribute('href')
            document_name = document_link.split('/')[-1]
            document_links.append(document_link)
            document_names.append(document_name)
            # download pdf
            download_pdf(document_link, document_name)
        else:
            # Get the help attribute which contains the links
            help_attribute = document_button.get_attribute('help')
            # Parse the help attribute as HTML
            soup = BeautifulSoup(help_attribute, 'html.parser')
            # Find all the links in the help attribute
            links = [a['href'] for a in soup.find_all('a', href=True)]
            # Download each link
            for link in links:
                document_name = link.split('/')[-1]
                document_link = f"https://emma.msrb.org{link}"
                document_links.append(document_link)
                document_names.append(document_name)
                # download pdf
                download_pdf(document_link, document_name)
        print('download_pdf done')
        # Click the plus button to reveal the CUSIP image
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//td[@class=' w30']/img[@class='detailsImg']"))
        )
        plus_button = driver.find_element(By.XPATH, "//td[@class=' w30']/img[@class='detailsImg']")
        plus_button.click()
        
        # Wait for the CUSIP image to be visible
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//td/a/img[@data-cusip]"))
        )
        
        print('cusip image found')
        # Extract and recognize text from CUSIP image
        cusip_img = driver.find_element(By.XPATH, "//td/a/img[@data-cusip]")
        img_url = cusip_img.get_attribute('src')
        cusip_text = get_cusip_text(img_url)

        print(cusip_text)
        return {
            'filing_type': filing_type,
            'disclosure_details': disclosure_details,
            'contact_info': contact_info,
            'document_links': document_links,
            'document_names': document_names,
            'cusip_text': cusip_text
        }

    except Exception as e:
        print(f"Error scraping detail page {link}: {e}")
        debug_page(f"{link.split('/')[-1]}_error")
        return None


def scrape_detail_pages_from_csv(file_path):
    df = read_csv(file_path)
    detail_data = []
    # set up tqdm progress bar
    pbar = tqdm(total=len(df), desc='Scraping detail pages', unit='rows')
    for index, row in df.iterrows():
        link = row['link']
        print('='*50)
        print('Sraping: ',link)
        driver = setup_driver()
        detail_info = scrape_detail_page(link)
        if detail_info:
            detail_data.append(detail_info)
        # save temp data
        detail_df = pd.DataFrame(detail_data)
        detail_df.to_csv(f'temp_detail_data_{file_path}.csv', index=False)
        driver.quit()
        # Optional: Add a delay to mimic human behavior and avoid detection
        random_sleep(2, 4)
        pbar.update(1)
    pbar.close()
    detail_df = pd.DataFrame(detail_data)
    return detail_df


In [57]:
# Initialize the WebDriver
# driver = setup_driver()

# Use the function to scrape details from the CSV
detail_df = scrape_detail_pages_from_csv('results_2024-04-24_2024-04-24.csv')
print(detail_df)

Scraping detail pages:   0%|          | 0/513 [00:00<?, ?rows/s]

Sraping:  https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21364921
No terms of use found
term_of_use handled
find body
clicked
part one done
Downloaded P21802482-P21383265-P21823263.pdf successfully.
download_pdf done
cusip image found
411890CL8


Scraping detail pages:   0%|          | 1/513 [00:11<1:42:00, 11.95s/rows]

Sraping:  https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21364924
No terms of use found
term_of_use handled
find body
clicked
part one done
Downloaded P21802487-P21383268-P21823267.pdf successfully.
download_pdf done
cusip image found
411890CL8


Scraping detail pages:   0%|          | 2/513 [00:25<1:48:15, 12.71s/rows]

...

Sraping:  https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21365019
No terms of use found
term_of_use handled
find body
clicked
Error scraping detail page https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21365019: Message: 

Debug info saved: scrape_detail_page_error_page_source.html and scrape_detail_page_error_screenshot.png


Scraping detail pages:   4%|▎         | 18/513 [05:10<3:07:28, 22.72s/rows]

Sraping:  https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21365065
No terms of use found
term_of_use handled
find body
clicked
Error scraping detail page https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21365065: Message: 
Stacktrace:
0   chromedriver                        0x0000000102d1a510 chromedriver + 4302096
1   chromedriver                        0x0000000102d12e58 chromedriver + 4271704
2   chromedriver                        0x000000010294419c chromedriver + 278940
3   chromedriver                        0x00000001029862c4 chromedriver + 549572
4   chromedriver                        0x00000001029bec5c chromedriver + 781404
5   chromedriver                        0x000000010297b004 chromedriver + 503812
6   chromedriver                        0x000000010297b9ec chromedriver + 506348
7   chromedriver                        0x0000000102ce2558 chromedriver + 4072792
8   chromedriver                        0x0000000102ce7004 chromedriver + 40919

Scraping detail pages:   4%|▎         | 19/513 [05:40<3:24:04, 24.79s/rows]

KeyboardInterrupt: 

In [139]:
detail_df.to_csv('test.csv', index=False)


In [74]:
scrape_detail_page('https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21364924')

No terms of use found
term_of_use handled
find body
clicked
disclosure_details: Rule 15c2-12 Disclosure
Audited Financial Statements or ACFR: Audited Financial Statement as of 12/31/2023-Hardin Co WD No 2, for the year ended 12/31/2023
Posted Date: 04/24/2024
Associated CUSIPs: 39
Issuer's Contact Information
Company: Hardin County Water District No. 2
Name: Mandy Isham
Address:
City, State, Zip:
Phone Number: 270-737-1056 ext. 259
Email: misham@hcwd2.org
Submitter's Contact Information
Company: Robert W Baird & Co., Inc.
Name: TAMMEY BIBB
Address: 500 W JEFFERSON ST
City, State, Zip: LOUISVILLE, KY 40202
Phone Number: 5025881124
Email: tbibb@rwbaird.com
part one done
Downloaded P21802487-P21383268-P21823267.pdf successfully.
download_pdf done
cusip image found
411890CL8


{'filing_type': 'Filing Type: FINANCIAL/OPERATING FILING (CUSIP-9 Based)',
 'disclosure_details': 'Rule 15c2-12 Disclosure\nAudited Financial Statements or ACFR: Audited Financial Statement as of 12/31/2023-Hardin Co WD No 2, for the year ended 12/31/2023\nPosted Date: 04/24/2024\nAssociated CUSIPs: 39',
 'contact_info': "Issuer's Contact Information\nCompany: Hardin County Water District No. 2\nName: Mandy Isham\nAddress:\nCity, State, Zip:\nPhone Number: 270-737-1056 ext. 259\nEmail: misham@hcwd2.org\nSubmitter's Contact Information\nCompany: Robert W Baird & Co., Inc.\nName: TAMMEY BIBB\nAddress: 500 W JEFFERSON ST\nCity, State, Zip: LOUISVILLE, KY 40202\nPhone Number: 5025881124\nEmail: tbibb@rwbaird.com",
 'document_links': ['https://emma.msrb.org/P21802487-P21383268-P21823267.pdf'],
 'document_names': ['P21802487-P21383268-P21823267.pdf'],
 'cusip_text': '411890CL8'}

In [27]:
driver = setup_driver()
driver.get('https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21338686')

# Handle the terms of use if it appears
handle_terms_of_use()
print('handled')
body = driver.find_element(By.TAG_NAME, 'body')
ActionChains(driver).move_to_element(body).click().perform()
# Wait for the detail page to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'card-body'))
)
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[@class=' w30']/img[@class='detailsImg']"))
)
plus_button = driver.find_element(By.XPATH, "//td[@class=' w30']/img[@class='detailsImg']")
plus_button.click()


# Wait for the CUSIP image to be visible
WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//td/a/img[@data-cusip]"))
)

print('cusip image found')
# Extract and recognize text from CUSIP image
cusip_img = driver.find_element(By.XPATH, "//td/a/img[@data-cusip]")
print(cusip_img.get_attribute('src'))


handled
cusip image found
https://emma.msrb.org/ImageGenerator.ashx?cusip9=A9EB5BB3326932130FBCEFABB8DBDEC64&rowNum=1&isLink=true


In [134]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/pdf',
        'Referer': 'https://emma.msrb.org/' 
    }

img_url = cusip_img.get_attribute('src')
img_response = requests.get(img_url, headers=headers, stream=True)
img = Image.open(BytesIO(img_response.content))
cusip_text = pytesseract.image_to_string(img, config='--psm 7').strip()

In [135]:
print(cusip_text)

054082AR3,
